## Estratégia Mista (exemplos)

In [1]:
using SymPy

# equações
function matriz_S(x) #função para gerar matriz de estratégias
    S = vec(collect(Base.product(reverse(x)...))) #produto cartesiano
    #transformar conjunto de estratégias em matriz
    S_M = []
    for i = 1:length(S[1])
        d = [y[i] for y in S]
        if i == 1 S_M = [d; ] else S_M = [S_M d] end
    end
    S_M = reverse(S_M, dims = 2)
    return S_M
end


function melhores_respostas(x,y) #S e U
ncol = size(x, 2) #numero de jogadores
BR = copy(x) #matriz contendo as melhores respostas
for i = 1:ncol #para cada agente i
    S_not_i = sum(x[:, 1:end .!= i], dims=2) # possiveis estratégias dos oponentes de i
    for j in unique(S_not_i) 
        k = findall(S_not_i .== j) #conjunto de estratégias contendo a estratégia j
        l = first.(Tuple.(k)) # linhas referentes à estratégia j
        h = findall(y[l,i] .== max(maximum(y[l,i]))) #estratégias do indivíduo i que maximisam sua utilidade frente à estrategia J
        #criar matriz BR
        for ll in l[h] BR[ll,i] = Sym(string(BR[ll,i],"^*")) end #assinalar melhores respostas com *
    end
end
return BR
end


# a soma das probabilidades para cada jogador deve ser iguala um
function sum_prob(x)
    eqn = []
    for i = 1:size(x, 2) 
        f = Eq(sum(unique(x[:,i])),1)
        if i == 1 eqn = [f; ] else eqn = [eqn f] end
    end
    return eqn
end


function hadamard_product(x)
    ncol = size(x, 2)
    d = []
    for i = 1:(ncol - 1)
        if i == 1 d = (x[:,i] .* x[:,(i+1)]) else d = (x[:,i+1] .* d) end   
    end    
    return d
end



function cpo(eqn1,eqn2,P)
    ee = []
    eqn3 = copy(eqn2)
    for pl in 1:size(P,2) # pl = jogador
        var = unique(P[:,pl])
        for i in 1:(size(var)[1]-1)
            for j in var 
                if j != i 
                    ee = eqn2[pl].subs.(j,solve(eqn1[pl],j)) 
                end
            end
            eq = Eq(diff.(ee[1],var[i]),0)
            eqn3[i,pl] = eq
        end
    end
    return eqn3
end



cpo (generic function with 1 method)

##### Ex: Batalha dos sexos
https://saylordotorg.github.io/text_introduction-to-economic-analysis/s17-03-mixed-strategies.html

correto

In [3]:
a1 = [Sym("a_$i^$j") for i=1 for j=["1","2"]]
a2 = [Sym("a_$i^$j") for i=2 for j=["1","2"]]
A = [a1 , a2]
S = matriz_S(A) 

#matriz de Probabilidades
p1 = [symbols("alpha_$i^$j", real = true, positive = true) for i=1 for j=["1","2"]]
p2 = [symbols("alpha_$i^$j", real = true, positive = true) for i=2 for j=["1","2"]]
P = [p1 , p2]
P = matriz_S(P) 

#matriz utilidade
U = [3 2;1 1;0 0;2 3]

4×2 Array{Int64,2}:
 3  2
 1  1
 0  0
 2  3

In [14]:
eqn1 = sum_prob(P)
S_P = hadamard_product(P)
eqn2 = S_P.T * U
eqn3 = cpo(eqn1,eqn2,P)
eqns = vec([eqn3 eqn1])
vars = unique(P)
solucao = solve(eqns,vars)

Dict{Any,Any} with 4 entries:
  alpha_2^2 => 3/4
  alpha_1^2 => 1/4
  alpha_2^1 => 1/4
  alpha_1^1 => 3/4

#### Matching pennies

https://saylordotorg.github.io/text_introduction-to-economic-analysis/s17-03-mixed-strategies.html

correto

In [7]:
U = [1 -1;-1 1;-1 1;1 -1]

eqn1 = sum_prob(P)
S_P = hadamard_product(P)
eqn2 = S_P.T * U
eqn3 = cpo(eqn1,eqn2,P)
eqns = vec([eqn3 eqn1])
vars = unique(P)
solucao = solve(eqns,vars)

Dict{Any,Any} with 4 entries:
  alpha_2^2 => 1/2
  alpha_1^2 => 1/2
  alpha_2^1 => 1/2
  alpha_1^1 => 1/2

#### Chicken

https://saylordotorg.github.io/text_introduction-to-economic-analysis/s17-03-mixed-strategies.html

correto

In [63]:
U = [0 0;-1 1;1 -1;-4 -4]

eqn1 = sum_prob(P)
S_P = hadamard_product(P)
eqn2 = S_P.T * U
eqn3 = cpo(eqn1,eqn2,P)
eqns = vec([eqn3 eqn1])
vars = unique(P)
sol = solve(eqns,vars)
sol = [Eq(var[1],var[2]) for var in sol] 

4-element Array{Sym,1}:
 α¹₁ = 3/4
 α²₁ = 1/4
 α¹₂ = 3/4
 α²₂ = 1/4

#### Welfare Game
Rasmusen (pp. 67)

Correto

In [62]:
U = [3 2;-1 3;-1 1;0 0]

eqn1 = sum_prob(P)
S_P = hadamard_product(P)
eqn2 = S_P.T * U
eqn3 = cpo(eqn1,eqn2,P)
eqns = vec([eqn3 eqn1])
vars = unique(P)
sol = solve(eqns,vars)
sol = [Eq(var[1],var[2]) for var in sol] 

4-element Array{Sym,1}:
 α¹₁ = 1/2
 α²₁ = 1/2
 α¹₂ = 1/5
 α²₂ = 4/5

#### Grab the Dollar

Rasmusen (pp. 73)

Correto

In [61]:
U = [-1 -1;1 0;0 1;0 0]

eqn1 = sum_prob(P)
S_P = hadamard_product(P)
eqn2 = S_P.T * U
eqn3 = cpo(eqn1,eqn2,P)
eqns = vec([eqn3 eqn1])
vars = unique(P)
sol = solve(eqns,vars)
sol = [Eq(var[1],var[2]) for var in sol] 

4-element Array{Sym,1}:
 α¹₁ = 1/2
 α²₁ = 1/2
 α¹₂ = 1/2
 α²₂ = 1/2

#### The Civic Duty Game

Rasmusen (pp. 80)

Correto

In [60]:
U = [0 0;10 7;7 10;7 7]

eqn1 = sum_prob(P)
S_P = hadamard_product(P)
eqn2 = S_P.T * U
eqn3 = cpo(eqn1,eqn2,P)
eqns = vec([eqn3 eqn1])
vars = unique(P)
sol = solve(eqns,vars)
sol = [Eq(var[1],var[2]) for var in sol] 

4-element Array{Sym,1}:
 α¹₁ = 3/10
 α²₁ = 7/10
 α¹₂ = 3/10
 α²₂ = 7/10

#### The Civic Duty Game (simbolic)

Rasmusen (pp. 77)

Correto

In [58]:
a,w,b,x,c,y,d,z = symbols("a,w,b,x,c,y,d,z", real = true)

U = [a w;b x;c y;d z]

eqn1 = sum_prob(P)
S_P = hadamard_product(P)
eqn2 = S_P.T * U
eqn3 = cpo(eqn1,eqn2,P)
eqns = vec([eqn3 eqn1])
vars = unique(P)
sol = solve(eqns,vars)
sol = [Eq(var[1],var[2]) for var in sol] 

4-element Array{Sym,1}:
  Eq(alpha_1^1, (y - z)/(-w + x + y - z))
 Eq(alpha_1^2, (-w + x)/(-w + x + y - z))
  Eq(alpha_2^1, (-b + d)/(a - b - c + d))
   Eq(alpha_2^2, (a - c)/(a - b - c + d))

#### Auditing Game I
Rasmusen (pp. 82)

In [59]:
c,f = symbols("c,f", real = true)

U = [4-c -f;4-c -1;0 0;4 -1]

eqn1 = sum_prob(P)
S_P = hadamard_product(P)
eqn2 = S_P.T * U
eqn3 = cpo(eqn1,eqn2,P)
eqns = vec([eqn3 eqn1])
vars = unique(P)
sol = solve(eqns,vars)
sol = [Eq(var[1] , var[2]) for var in sol] 

4-element Array{Sym,1}:
       Eq(alpha_1^1, 1/f)
 Eq(alpha_1^2, (f - 1)/f)
       Eq(alpha_2^1, c/4)
   Eq(alpha_2^2, 1 - c/4)